In [ ]:
library(data.table)

On and around page 304 of _Field Experiments_ Green & Gerber give us a great example to work with to consider multi-factor experiments -- the case of "Colin" and "Jose" sending letters to their elected representatives. 

> This is actually a classic example of an audit experiment -- see for example the work of [Devah Pager](https://scholar.harvard.edu/files/pager/files/pager_ajs.pdf) who used audit studies to evalute the penalty suffered by black job applicants in the labor market. 

The thing about the example is that Green and Gerber have provided us the data in a way that it has already been summarized for us. This is both (a) common to receive a table that has already been summarized; and (b) frustrating because we can't actually do any work against the data. 

Not only have Green and Gerber summarized the data for us, but they have presented it in four columns and a single row. This hides the very clear 2x2 structure of the experiment! 

Write out the 4x1 table as a 2x2 table in the markdown cell below

|             | Colin | Jose | Total |
|-------------|-------|------|-------|
| Good Grammar|       |      |       |
| Bad Grammar |       |      |       |
| **Total**   |       |      |       |

After writing down what the conditions are, can you also make data that would produce this table? 

- What is the outcome that they are evaluating, and what are the possible values for this outcome? How should you encode this information? 
- How many observations are there for each of the cells in this experiment? 
- How can you represent the percentages that you've written down in each cell? 

In [ ]:
d <- data.table(
    'id'      = 1:N, # change N here to the total number of observations,
    'name'    = rep(), # fill in with name info 
    'grammar' = rep(), # fill in with grammar info
    ) 

Once you've got the experiment structured, then you can start to build out the outcome you identified above. We'll create a variable name -- `response` -- for you and set all the values to 0. 

Select rows of the data.table, and fill in the response variable as appropriate to complete the table. 

In [ ]:
d[ , response := 0L]

If you've built your data correctly, then you should be able to run the next line, and return back out the 2x2 table that you wrote down earlier. 

In [ ]:
d[ , .(response_rate = mean(response)), by = .(name, grammar)]

> In the check above, you are creating a new variable, called `response_rate` that is the avarage of the responses within each of the name-grammar conditions. Those name-grammar conditions are called out in the `... by = .(name, grammar)` argument. 

What you're showing above is equivalent to estimating a "saturated" model where you have as many indicators for the condition as you do have conditions. These saturated models will report the means within each of the condition groups. 

One way to estimate a saturated model is to suppress the intercept -- using a -1 call (read, "not intercept") -- and then include the indicators for the features. For example, 

In [ ]:
saturated_model_1 = d[ , lm(response ~ -1 + name + grammar)]
summary(saturated_model_1)

The other way to estimate this saturated model with a regression is to include an indicator for each of the name-grammar conditions. 

Produce a new column on this data.table called `treatment_condition` that has an indicator for the name-grammar condition, and then use this single feature in a model to estimate the saturated indicator. (Don't just hard code this based on position in the data.table; instead, think about writing the vectorized conditional operation.) 

In [ ]:
d[name=='' & grammar=='', treatment_condition := ]

In [ ]:
saturated_model_2 = d[ , lm(response ~ treatment_condition)]

Take care to interpret what you're getting back from this model.

1. Does it produce the results that you thought it would? Why or why not? If not, what change could you make to the model to make it produce what you anticipated it would? 
2. *Specifically* what tests are being reported? What is the null hypothesis for each of these tests? Is this a meaningful test? Why or why not? 

# Estimate Model 9.12

Despite the ease of estimating the *saturated model*, it is possible to structure your model so that the sets of tests reported are more useful to you. In the model described as model 9.12 in _Field Experiments_ Green & Gerber write down a model that is: 

$$
Y_{i} = \beta_{0} + \beta_{1} Name + \beta_{2} Gramar + \beta_{3} Name \times Grammar
$$ 

Estimate this model in the next cell and store the results as an object `interaction_model`. 

In [ ]:
interaction_model = d[ , lm()]
interaction_model

Inspect the results of this model: 
1. What are each of the coefficients reporting? 
2. Which coefficient or coefficients in this model have an exactly identical coefficient in the `saturated_model`. What does this mean?  Why are the other coefficients different?
3. How can you combine sets of coefficients to contain the same information as in the earlier model? 
4. What are the tests that are being reported for each of these coefficients? 
  1. Is the null hypothesis in this model for each of the coefficients the same as before? Why or why not? 
  2. Are you surprised that fewer coefficients are significant in this model? Why or why not? 
  3. Which set of null hypotheses -- those in the `saturated_model` or in this `interaction_model` more closely match with the tests that you care about? Why? 